In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os.path
from bs4 import BeautifulSoup
import requests

In [2]:
first_record = 10000
last_record = 22317
record_segment_file_name = '../../../2_data/' + 'record_segment_good_length_' + str(first_record) + '_' + str(last_record - 1) +'.csv'

In [3]:
record_segment_file_name

'../../../2_data/record_segment_good_length_10000_22316.csv'

In [4]:
record_path_local = '../../../2_data/record_path_mimic3_matched.csv'
record_root_url = 'https://physionet.org/content/mimic3wdb/1.0/matched'
record_path_url = record_root_url + '/RECORDS-waveforms'

signal_extract_name = 'PLETH'
sampling_frequency = 125 # Hz
signal_extract_minimum_length = 5 # the minimum length of the signal segment, in minutes

print('record_path_local: {}'.format(record_path_local))
print('record_root_url: {}'.format(record_root_url))
print('record_path_url: {}'.format(record_path_url))

record_path_local: ../../../2_data/record_path_mimic3_matched.csv
record_root_url: https://physionet.org/content/mimic3wdb/1.0/matched
record_path_url: https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


In [5]:
def get_url_content(url, tag=None):
    
    '''
    This function scrapes a list of useful information from a given PhysioNet URL.
    If the URL address points to an HTML document, the information to be extracted is define by a tag.
    I found this address (https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) usefull
      for details on how to scrape a web page
    '''
    
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    if '<!DOCTYPE html>' in str(soup):
        content = getattr(soup, str(tag)).getText()
        #content = soup.pre.getText()
    else:
        content = soup.getText()
    return content.splitlines()

In [6]:
def get_signal_name(url_content):
    return [x.split(' ')[-1] for x in url_content[1:]]

In [7]:
tag = 'pre'
record_path_list = get_url_content(record_path_url, tag)
print(len(record_path_list))
for i in range(5):
    print('record_path: {}'.format(record_path_list[i]))

22317
record_path: p00/p000020/p000020-2183-04-28-17-47
record_path: p00/p000030/p000030-2172-10-16-12-22
record_path: p00/p000033/p000033-2116-12-24-12-35
record_path: p00/p000033/p000033-2116-12-25-13-11
record_path: p00/p000052/p000052-2191-01-10-02-21


In [ ]:
tag = 'pre'
#last_record = len(record_path_list) + 1
for process_record_index in range(first_record, last_record):
        
    #print(process_record_index)
    record_path = record_path_list[process_record_index]
    record_path_short = record_path.split('/')[2]
    print('processing record #: {}, record_path_short: {}'.format(process_record_index, record_path_short))
    
    directory = str(record_path.split("/")[0]) + '/' + str(record_path.split("/")[1])
    #print('directory: {}'.format(directory))
            
    master_waveform_header = record_root_url + '/' + record_path + '.hea'
    #print('master_waveform_header: {}'.format(master_waveform_header))
    
    target_url = master_waveform_header
    master_waveform_content = get_url_content(target_url, tag)
    #print('master_waveform_content: {}'.format(master_waveform_content))
    #print('       ...........\n')
    
    master_waveform_content_first_line = master_waveform_content[0].split(' ')
    record_length = int(master_waveform_content_first_line[3]) / sampling_frequency
    #print('record_length (s): {}'.format(record_length))
    #print('       ...........\n')
        
    waveform_layout_header = master_waveform_content[1].split(' ')[0]
    #print('1. waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    waveform_layout_header = waveform_layout_header + '.hea'
    #print('2. waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    waveform_layout_header = record_root_url + '/' + directory + '/' + waveform_layout_header
    #print('record_root_url: {}'.format(record_root_url))
    #print('directory: {}'.format(directory))
    #print('3. waveform_layout_header: {}\n'.format(waveform_layout_header))
    #print('waveform_layout_header: {}\n'.format(waveform_layout_header))
    
    target_url = waveform_layout_header
    waveform_layout_content = get_url_content(target_url, tag)
    #print('waveform_layout_content: {}'.format(waveform_layout_content))
        
    record_signal_names = get_signal_name(waveform_layout_content)
    #print('record_signal_names: {}'.format(record_signal_names))
    
    if signal_extract_name in record_signal_names and record_length > 60 * signal_extract_minimum_length: # and len(record_segments_path_list):      
        
        # Open the record_segment_file_name file with access mode 'a'
        record_segment_file_object = open(record_segment_file_name, 'a')

        # get the waveform record name
        waveform_record_id = [x.split(' ')[0].split('_') for x in master_waveform_content if '3' == x[0]][0][0]
        #print('waveform_record_id: {}\n'.format(waveform_record_id))
        
        # get the waveform_record_state
        waveform_record_state = [(waveform_record_id + '_' in x) and not (waveform_record_id + '_layout' in x) 
                for x in master_waveform_content]
        #print('waveform_record_state: {}\n'.format(waveform_record_state))

        # get the waveform_record_segment
        waveform_record_segment = [master_waveform_content[x].split(' ') for x, y in enumerate(waveform_record_state) if y]
        #print('waveform_record_segment: {}\n'.format(waveform_record_segment))
      
        for segment_index in range(len(waveform_record_segment)):
            # number of samples: int(waveform_record_segment[segment_index][1])
            # sampling frequency: int(master_waveform_content_first_line[2])
            # length (in seconds) of the record segment: number of samples / sampling frequency
            
            # https://physionet.org/content/mimic3wdb/1.0/matched/p00/p000107/3168852_0001.hea
            
            segment_header = record_root_url + '/' + directory + '/' + waveform_record_segment[segment_index][0] + '.hea'
            #print(segment_header)
            
            segment_header_content = get_url_content(segment_header, tag)
            #print('segment_header_content: {}\n'.format(segment_header_content))
            
            segment_signal_names = get_signal_name(segment_header_content)
            #print('segment_signal_names: {}\n'.format(segment_signal_names))
           
            waveform_record_segment_length = int(waveform_record_segment[segment_index][1]) / sampling_frequency
            #print('waveform_record_segment_length: {}\n'.format(waveform_record_segment_length))
            
            if signal_extract_name in segment_signal_names and waveform_record_segment_length > 60 * signal_extract_minimum_length:
                
                waveform_record_segment_good_length = waveform_record_segment[segment_index][0]
                print('  ' + str(process_record_index) + ',' + record_path + ',' + waveform_record_segment_good_length)                
                
                # Append the waveform_record_segment_good_length_root at the end of file record_segment_file_name
                record_segment_file_object.write(str(process_record_index) + ',' + record_path + ',' + waveform_record_segment_good_length + '\n')
        
        # Close the file record_segment_file_name
        record_segment_file_object.close()
        
    #index_of_last_record_processed = index_of_last_record_processed + 1
    print('  \nindex of last record processed successfully: {}'.format(process_record_index))  
    print('--------------------------------------')

    
    


processing record #: 10000, record_path_short: p049022-2160-12-03-23-50
  10000,p04/p049022/p049022-2160-12-03-23-50,3198728_0003
  10000,p04/p049022/p049022-2160-12-03-23-50,3198728_0005
  10000,p04/p049022/p049022-2160-12-03-23-50,3198728_0007
  
index of last record processed successfully: 10000
--------------------------------------
processing record #: 10001, record_path_short: p049023-2179-09-03-21-42
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0002
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0005
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0010
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0012
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0015
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0019
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0023
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0025
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0031
  10001,p04/p049023/p049023-2179-09-03-21-42,3708030_0036
  10001,p

  10014,p04/p049053/p049053-2149-05-24-05-42,3539321_0006
  10014,p04/p049053/p049053-2149-05-24-05-42,3539321_0007
  10014,p04/p049053/p049053-2149-05-24-05-42,3539321_0010
  
index of last record processed successfully: 10014
--------------------------------------
processing record #: 10015, record_path_short: p049058-2178-01-14-01-45
  10015,p04/p049058/p049058-2178-01-14-01-45,3292980_0003
  10015,p04/p049058/p049058-2178-01-14-01-45,3292980_0004
  10015,p04/p049058/p049058-2178-01-14-01-45,3292980_0005
  10015,p04/p049058/p049058-2178-01-14-01-45,3292980_0008
  10015,p04/p049058/p049058-2178-01-14-01-45,3292980_0010
  
index of last record processed successfully: 10015
--------------------------------------
processing record #: 10016, record_path_short: p049067-2112-08-18-16-22
  10016,p04/p049067/p049067-2112-08-18-16-22,3711977_0009
  10016,p04/p049067/p049067-2112-08-18-16-22,3711977_0010
  10016,p04/p049067/p049067-2112-08-18-16-22,3711977_0012
  10016,p04/p049067/p049067-2112

  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0011
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0017
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0020
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0023
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0026
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0029
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0032
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0035
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0038
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0044
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0053
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0056
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0062
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0065
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0074
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0086
  10030,p04/p049168/p049168-2168-07-18-20-48,3165402_0095
  10030,p04/p0

  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0001
  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0003
  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0005
  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0007
  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0009
  10033,p04/p049190/p049190-2117-02-26-19-20,3868478_0011
  
index of last record processed successfully: 10033
--------------------------------------
processing record #: 10034, record_path_short: p049191-2117-06-29-14-03
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0002
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0004
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0006
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0007
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0009
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0011
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0013
  10034,p04/p049191/p049191-2117-06-29-14-03,3745945_0015
  10034,p04/p049191/p04

  
index of last record processed successfully: 10052
--------------------------------------
processing record #: 10053, record_path_short: p049295-2121-06-11-11-10
  
index of last record processed successfully: 10053
--------------------------------------
processing record #: 10054, record_path_short: p049304-2148-09-11-19-07
  10054,p04/p049304/p049304-2148-09-11-19-07,3295346_0027
  
index of last record processed successfully: 10054
--------------------------------------
processing record #: 10055, record_path_short: p049311-2121-05-06-01-03
  10055,p04/p049311/p049311-2121-05-06-01-03,3316852_0005
  10055,p04/p049311/p049311-2121-05-06-01-03,3316852_0006
  10055,p04/p049311/p049311-2121-05-06-01-03,3316852_0008
  
index of last record processed successfully: 10055
--------------------------------------
processing record #: 10056, record_path_short: p049311-2121-05-06-16-22
  10056,p04/p049311/p049311-2121-05-06-16-22,3627342_0003
  10056,p04/p049311/p049311-2121-05-06-16-22,36273

...

  14199,p06/p065824/p065824-2132-09-22-19-16,3186294_0050
  14199,p06/p065824/p065824-2132-09-22-19-16,3186294_0056
  14199,p06/p065824/p065824-2132-09-22-19-16,3186294_0068
  14199,p06/p065824/p065824-2132-09-22-19-16,3186294_0071
  
index of last record processed successfully: 14199
--------------------------------------
processing record #: 14200, record_path_short: p065824-2132-09-23-17-12
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0002
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0005
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0008
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0011
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0014
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0017
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0020
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0022
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0025
  14200,p06/p065824/p065824-2132-09-23-17-12,3217138_0028
  14200,p06/p065824/p06

  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0050
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0054
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0056
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0058
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0059
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0062
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0064
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0066
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0070
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0074
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0076
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0080
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0085
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0088
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0090
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0092
  14210,p06/p065837/p065837-2104-11-11-16-47,3114472_0096
  14210,p06/p0

  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0042
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0045
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0048
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0051
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0054
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0057
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0060
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0063
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0066
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0069
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0072
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0075
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0076
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0079
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0087
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0090
  14224,p06/p065890/p065890-2192-06-15-12-29,3926255_0093
  
index of la

  
index of last record processed successfully: 14237
--------------------------------------
processing record #: 14238, record_path_short: p065925-2112-05-16-14-22
  14238,p06/p065925/p065925-2112-05-16-14-22,3653740_0033
  14238,p06/p065925/p065925-2112-05-16-14-22,3653740_0041
  
index of last record processed successfully: 14238
--------------------------------------
processing record #: 14239, record_path_short: p065925-2112-05-30-13-08
  14239,p06/p065925/p065925-2112-05-30-13-08,3637163_0002
  14239,p06/p065925/p065925-2112-05-30-13-08,3637163_0004
  14239,p06/p065925/p065925-2112-05-30-13-08,3637163_0006
  14239,p06/p065925/p065925-2112-05-30-13-08,3637163_0009
  14239,p06/p065925/p065925-2112-05-30-13-08,3637163_0010
  
index of last record processed successfully: 14239
--------------------------------------
processing record #: 14240, record_path_short: p065925-2112-06-01-15-42
  14240,p06/p065925/p065925-2112-06-01-15-42,3291524_0003
  14240,p06/p065925/p065925-2112-06-01-15

  14244,p06/p065951/p065951-2173-05-26-13-01,3946597_0303
  
index of last record processed successfully: 14244
--------------------------------------
processing record #: 14245, record_path_short: p065956-2158-11-10-22-29
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0004
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0008
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0010
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0013
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0014
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0019
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0026
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0032
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0034
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0042
  14245,p06/p065956/p065956-2158-11-10-22-29,3300246_0044
  
index of last record processed successfully: 14245
--------------------------------------
processing record #: 14246, record_path_short:

In [ ]:
waveform_record_segment[segment_index][0]

In [ ]:
waveform_record_segment[segment_index][1]

In [ ]:
master_waveform_content_first_line

In [ ]:
master_waveform_content_first_line[2]

In [ ]:
int(master_waveform_content_first_line[2])